<a href="https://colab.research.google.com/github/rhaagsma/Projeto_Final_Marvel/blob/main/ProjetoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazendo Importações e passando os parâmetros necessários para realizar as resquisições

In [2]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd
import ast

public_key = userdata.get('public_key')
private_key = userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest
    }

In [ ]:
url1 = "https://gateway.marvel.com/v1/public/characters"
url2 = "https://gateway.marvel.com/v1/public/comics"

Definição da Função que retorna os dados da Requisição em um Dataframe

In [ ]:
def get_data(url):
  total_data = 0
  offset = 0
  data = pd.DataFrame()

  # Garante a entrada no loop na primeira vez
  while offset < total_data or total_data == 0:
      print(offset)
      auth_params['offset'] = offset
      response = requests.get(url, auth_params)
      response = response.json()

      # Atualiza total_data após a primeira requisição
      if total_data == 0:
          total_data = response['data']['total']

      # Verifica se a resposta contém dados
      if not response['data']['results']:
          break

      response = pd.json_normalize(response['data']['results'])
      df1 = response
      data = pd.concat([data, df1])
      offset += len(df1)

  return data

Requisição do Endpoint Characters e inserção dos dados no Banco

In [ ]:
characters = get_data(url1)

conn = sqlite3.connect('banco.db')
columns = ['id', 'name','description']

characters[columns].to_sql('characters', conn, if_exists='replace', index=False)

conn.close()

Definição de uma Função que extrai os preços da lista de dicionários do Endpoint Comics

In [24]:
def extrair_preco(lista_precos):
    try:
        # Converte a string em lista de dicionários, se necessário
        if isinstance(lista_precos, str):
            lista_precos = ast.literal_eval(lista_precos)

        for item in lista_precos:
            if item.get('type') == 'printPrice':
                return item.get('price')
    except Exception as e:
        print(f"Erro ao processar: {lista_precos} -> {e}")

    return 0


Requisição do Endpoint Comics e inserção dos dados no Banco

In [27]:
comics = get_data(url2)

# Expressão regular para encontrar um ano de 4 dígitos entre parênteses
comics['year'] = comics['title'].str.extract(r'\((\d{4})\)').fillna(0).astype(int)
comics['price'] = comics['prices'].apply(extrair_preco)

conn = sqlite3.connect('banco.db')
columns = ['id','digitalId','title','year','pageCount', 'price']

comics[columns].to_sql('comics', conn, if_exists='replace', index=False)

conn.close()

Requisição para criar a Tabela Associativa de Characters e Comics no Banco

In [ ]:
#query para acessar os ids dos personagens
conn = sqlite3.connect('banco.db')

query = "SELECT id FROM characters"
df = pd.read_sql_query(query, conn)

conn.close()

request_count = 0
comics_chars = pd.DataFrame()
limit = 100

for index, row in df.iterrows():
  char_id = row['id']
  print(index)
  offset = 0
  total = 0

  # Loop enquanto o offset for menor que o total ou total ainda não foi definido
  while offset <= total or total == 0:
      comic_params = auth_params
      comic_params.update({
          'offset': offset
      })

      comic_resp = requests.get(f"https://gateway.marvel.com/v1/public/characters/{char_id}/comics", params=comic_params)
      request_count += 1

      comics_data = comic_resp.json()
      results = comics_data['data']['results']

      temp_df = pd.DataFrame(results)

      temp_df['character_id'] = char_id

      comics_chars = pd.concat([comics_chars, temp_df], ignore_index=True)

      if total == 0:
          total = comics_data['data']['total']
          if total == 0:
            break

      if total < limit:
        offset += total
      else:
        offset += limit

In [ ]:
conn = sqlite3.connect('banco.db')

comics_chars.rename(columns={'id': 'comic_id'})
columns = ['character_id', 'comic_id']

comics_chars[columns].to_sql('characters_comics', conn, if_exists='replace', index=False)

conn.close()